In [24]:
import sys

# Import
import numpy as np
import pandas as pd
from keras.src.callbacks import EarlyStopping
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [25]:
# Set random seeds
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# Combine datasets

In [26]:
# Read the datasets
real_images = pd.read_csv("real_images.csv")
fake_images = pd.read_csv("fake_images.csv")

In [27]:
# Normalize values between 0 and 1
real_images /= 255.
fake_images /= 255.

In [28]:
# Add label column
real_images["label"] = 1.0
fake_images["label"] = 0.0

In [29]:
# Combine the two datasets
images = pd.concat([real_images, fake_images], ignore_index=True)

In [30]:
# Shuffle
images = images.sample(frac=1).reset_index(drop=True)

In [31]:
# Split into training and test sets
features = images.drop("label", axis=1)
labels = images["label"]

# Create discriminator models

In [34]:
if sys.platform.startswith('darwin'):
    optimizer = tf.keras.optimizers.legacy.SGD
else:
    optimizer = "sgd"
    
TOTAL_MODELS = 10
TRAINING_SIZE = 20_000
TEST_SIZE = 5_000
MODEL_TRAINING_SIZE = int(TRAINING_SIZE / TOTAL_MODELS)
MODEL_TEST_SIZE = int(TEST_SIZE / TOTAL_MODELS)

x_test, x_train = features[TRAINING_SIZE:], features[:TRAINING_SIZE]
y_test, y_train = labels[TRAINING_SIZE:], labels[:TRAINING_SIZE]

for i in range(TOTAL_MODELS):
    print(i)
    # create training and test set for the model
    start_idx = i * MODEL_TRAINING_SIZE
    end_idx = (i + 1) * MODEL_TRAINING_SIZE

    m_x_test = features[end_idx:end_idx + MODEL_TEST_SIZE]
    m_x_train = features[start_idx:end_idx]

    m_y_test = labels[end_idx:end_idx + MODEL_TEST_SIZE]
    m_y_train = labels[start_idx:end_idx]
    # Create model
    model = keras.models.Sequential([
        keras.layers.Input(shape=[784]),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid")
    ])
    # Compile model
    model.compile(loss="binary_crossentropy",
                  optimizer="sgd",
                  metrics=["accuracy"])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    # Train model
    model.fit(m_x_train, m_y_train, epochs=5,
              validation_data=(m_x_test, m_y_test))
    # Save model
    model.save(f"discriminator_models/discriminator_{i+1}.keras")

0
Epoch 1/5
63/63 [==============================] - 0s 2ms/step - loss: 0.4875 - accuracy: 0.9655 - val_loss: 0.3870 - val_accuracy: 1.0000
Epoch 2/5
63/63 [==============================] - 0s 935us/step - loss: 0.3499 - accuracy: 1.0000 - val_loss: 0.3130 - val_accuracy: 1.0000
Epoch 3/5
63/63 [==============================] - 0s 884us/step - loss: 0.2923 - accuracy: 1.0000 - val_loss: 0.2658 - val_accuracy: 1.0000
Epoch 4/5
63/63 [==============================] - 0s 900us/step - loss: 0.2510 - accuracy: 1.0000 - val_loss: 0.2293 - val_accuracy: 1.0000
Epoch 5/5
63/63 [==============================] - 0s 877us/step - loss: 0.2177 - accuracy: 1.0000 - val_loss: 0.1993 - val_accuracy: 1.0000
1
Epoch 1/5
63/63 [==============================] - 0s 2ms/step - loss: 0.5093 - accuracy: 0.9265 - val_loss: 0.4253 - val_accuracy: 1.0000
Epoch 2/5
63/63 [==============================] - 0s 951us/step - loss: 0.3629 - accuracy: 1.0000 - val_loss: 0.3504 - val_accuracy: 1.0000
Epoch 3/5
63/